# US Air Freight and Waterway Volumes

---

A look at air and domestic waterway freight volumes, according to the US DOT Bureau of Transportation Statistics.

In [14]:
import pandas as pd
import altair as alt
import re

In [15]:
# https://www.transtats.bts.gov/osea/seasonaladjustment/

af_raw = !curl 'https://transtats.bts.gov/osea/seasonaladjustment/?PageVar=AIR_RTMFM' | grep 'series:'

df_af = pd.DataFrame.from_records(re.findall("'(\d\d/\d\d/\d\d\d\d \d\d:\d\d:\d\d)'\), y: (\d+)", "".join(af_raw)),
                                  columns=["date", "tonMiles"])

df_af['dt'] = df_af.date.apply(pd.to_datetime)

#df_af.head()

## US Airline Freight

In [16]:
alt.Chart(df_af).mark_line().encode(
    alt.X('dt:T', axis=alt.Axis(title='')),
    alt.Y('tonMiles:Q', axis=alt.Axis(title='Volume [ton-miles]'))
).properties(
    title='US Air Freight Volume',
    width=700,
    height=400
)

alt.Chart(...)

In [17]:
df_af['change'] = df_af.set_index('dt')['tonMiles'].apply(int).pct_change(12).apply(lambda v: v * 100).values

c = alt.Chart(df_af).mark_bar(size=2).encode(
    alt.X('dt:T', axis=alt.Axis(title='')),
    alt.Y('change:Q', axis=alt.Axis(title='Year-over-Year Change [%]')),
    tooltip=[alt.Tooltip('dt:T', title='Date'), alt.Tooltip('change:Q', title='% Change', format=',.01f')],
    color=alt.condition(f"datum.change < 0",
        alt.value('lightsalmon'),
        alt.value('royalblue')
    ),
).properties(
    title='US Air Freight Volume Growth',
    width=750,
    height=400
)

c.save('transportation-air-waterway.png')
c.display()

alt.Chart(...)

## Internal US Waterways Shipping

In [18]:
wwf_raw = !curl 'https://transtats.bts.gov/osea/seasonaladjustment/?PageVar=WATERBORNE' | grep 'series:'

df_wwf = pd.DataFrame.from_records(re.findall("'(\d\d/\d\d/\d\d\d\d \d\d:\d\d:\d\d)'\), y: (\d+)", "".join(wwf_raw)),
                                  columns=["date", "megaShortTons"])

df_wwf['dt'] = df_wwf.date.apply(pd.to_datetime)

#df_wwf.head()

In [19]:
alt.Chart(df_wwf).mark_line().encode(
    alt.X('dt:T', axis=alt.Axis(title='')),
    alt.Y('megaShortTons:Q', axis=alt.Axis(title='Volume [Million short tons]'))
).properties(
    title='US Internal Waterways Shipping Volume',
    width=700,
    height=400
)

alt.Chart(...)

In [20]:
df_wwf['change'] = df_wwf.set_index('dt')['megaShortTons'].apply(int).pct_change(12).apply(lambda v: v * 100).values

alt.Chart(df_wwf).mark_bar(size=2).encode(
    alt.X('dt:T', axis=alt.Axis(title='')),
    alt.Y('change:Q', axis=alt.Axis(title='Year-over-Year Change [%]')),
    tooltip=[alt.Tooltip('dt:T', title='Date'), alt.Tooltip('change:Q', title='% Change', format=',.01f')],
    color=alt.condition(f"datum.change < 0",
        alt.value('lightsalmon'),
        alt.value('royalblue')
    ),
).properties(
    title='US Internal Waterways Volume Growth',
    width=750,
    height=400
)

alt.Chart(...)